In [ ]:
############### GET API KEYS
from twython import Twython
import json
import csv

credentials = {}
credentials["API_KEY"] = "AZVjw2iplhA5dNszUeGxjOgjv"
credentials["API_SECRET"] = "RZ8E6R59gC9oW92Qz4YFRgygACvD4onQRzqyUTPRzKfnnQSW3H"
credentials["ACCESS_TOKEN"] = "1501925600105218050-7xtRvXQGeA6wkft2kPoA5JOj0PnBmq"
credentials["ACCESS_TOKEN_SECRET"] = "0Fl3hzhnogVszJGEorAh9r7XBRspU7XQaWERfC8JXtA2M"


##### SAVE CREDENTIALS IN JSON FILE
with open("credentials.json", "w") as file:
    json.dump(credentials, file)








In [ ]:
# AUTHENTICATION OF TWITTER TOKENS OAUTH
from twython import Twython
import json


########## GET TOKENS FROM THE SAVED JSON FILE
credentials = {}
with open("credentials.json", "r") as file:
    credentials = json.load(file)

twitter = Twython(app_key=credentials["API_KEY"],
                 app_secret=credentials["API_SECRET"], 
                 oauth_token=credentials["ACCESS_TOKEN"],
                 oauth_token_secret=credentials["ACCESS_TOKEN_SECRET"])


In [ ]:
################ UTILITY CLASS TO PROCESS TWEETS SAVING

class Twitter_Utility:

    def __init__(self):
        self.results = {"statuses":[]}
        self.header_counter = 0
    
    ############### EXTRACT INDIVIDUAL TWEET ATTRIBUTES
    def process_tweet(self, data):
        tweet = {}
        tweet["created_at"] = data["created_at"]
        tweet["location"] = data["user"]["location"]
        tweet["followers_count"] = data["user"]["followers_count"]
        tweet["verified"] = data["user"]["verified"]
        tweet["text"] = data["text"]
        tweet["client_platform"] = data["source"]
        tweet["hashtags"] = [hashtag["text"] for hashtag in data["entities"]["hashtags"]]
        tweet["retweet_count"] = data["retweet_count"]
        # tweet["favorited"] = data["favorited"]
        tweet["favorite_count"] = data["favorite_count"]
        if "place" in data and data["place"] != None:
            tweet["long"] = data["place"]["bounding_box"]["coordinates"][0][0][0]
            tweet["lat"] = data["place"]["bounding_box"]["coordinates"][0][0][1]
        else:          
            tweet["long"] = None
            tweet["lat"] = None

        return tweet


    ######### SAVE THE CSV COLUMN NAMES FROM THE PROCESSED TWEETS KEYS
    def csv_set_header(self, tweet):
        if self.header_counter == 0:
          with open("saved_tweets.csv", "a", encoding="utf-8",newline="") as file:
            header = [key for key in self.process_tweet(tweet)]
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)
            self.header_counter = 1
        
    ######### FUNCTIN TO SAVE TWEEETS
    def save_to_csv(self, tweet):
            with open("saved_tweets.csv", "a", encoding="utf-8", newline="") as file:
                csv_writer = csv.writer(file)
                tweet_content = [tweet[key] for key in tweet]
                csv_writer.writerow(tweet_content)
                return tweet_content

    ########### FUNCTION TO SEARCH TWEETS BY USING GIVEN  QUERY
    def search_tweet(self, search_query,count,lang="en",result_type="mixed"):
        for i in range(49):
            results = twitter.search(q=search_query, count= count,lang=lang, result_type=result_type)   
            self.results["statuses"].extend(results["statuses"])
        # self.results["statuses"] = self.results["statuses"].append(results["statuses"])
        # self.results["statuses"] = self.results["statuses"][0]
        # self.results["statuses"] = self.results["statuses"][0]

        
        #print("multi list", self.results)
        return self.results
        
    #### FUNCTION TO PROCESS WHOLE FETCHING & SAVING OF TWEETS 
    def save_tweets(self):
        if self.results.get("statuses"):
                #self.csv_set_header(self.results["statuses"][0])
                for result in self.results["statuses"]:
                    tweet = self.process_tweet(result) 
                    self.save_to_csv(tweet)




twitter_util = Twitter_Utility()

In [ ]:
# TWITTER USA TRENDS FETCHING
import pandas as pd

######### UTILITY FUNCTION TO CONSTRUCT THE API URL OF TRENDS FOR A PARTICULAR LOCATTION
def twitter_trends_url(url, id):
    url = twitter.construct_api_url(api_url=url, id = id)
    print(url)
    return url



######################### WOEIDS OF DIFFERENT LOCATIONS
# WOEID 
# UK=23424975 
# USA=23424977
# WORLDWIDE=1
# Birmingham=12723


response = twitter._request(url= twitter_trends_url("https://api.twitter.com/1.1/trends/place.json",id=23424977),
                    json_encoded=True,
                    method="GET")


############# SAVE TRENDS INTO JSON
with open("worldtrends.json", "w") as file:
        json.dump(response, file)


top_15_trends_list = list(response[0]["trends"])[0:15]

top_15_trends_list

#################### CREATE DATAFRAME COLUMNS FOR TRENDS
df = pd.DataFrame(columns=["name","tweet_volume","url","query"])


# https://stackoverflow.com/questions/10715965/create-a-pandas-dataframe-by-appending-one-row-at-a-time


############ FOR EVERY TREND SAVE ITS CONTENT INTO RESPECTIVE LOCATION
for index in range(len(top_15_trends_list)):
    df.loc[index] = [top_15_trends_list[index]["name"], 
                        top_15_trends_list[index]["tweet_volume"], 
                        top_15_trends_list[index]["url"],
                        top_15_trends_list[index]["query"]
                        ] 


df

In [ ]:
#################### BARPLOT FOR USA TRENDS

import seaborn as sns
import matplotlib.pyplot as plt

############ DROP ANY COLUMNS THAT DON"T HAVE ANY TWEET VOLUME OR TWEETS NUMBERS
df = df.dropna()

plt.figure(figsize=(10,6))

##### SORT THE DATAFRAME TO SHOW TOP TRENDS FROM TOP TO BOTTOM
sorted_df = df.sort_values(['tweet_volume'], ascending=False).reset_index(drop=True)


###### PLOT THE BARPLOT OF TRENDING TOPICS
sns.barplot(data=sorted_df,
             x="name", 
             y="tweet_volume",
             order=sorted_df.name)
plt.xlabel("USA Twitter Trends", size= 15)             
plt.ylabel("Tweet Count", size= 15)             
plt.xticks(rotation=90)
plt.show()







In [ ]:

top_trend_on_twitter_query = sorted_df.iat[0,0]
trend_tweets = twitter_util.search_tweet(top_trend_on_twitter_query,count=100)


# SHOW THE QUERY OF TOP TREND USED TO FETCH TWEETS

print(top_trend_on_twitter_query)
twitter_util.save_tweets()


In [ ]:
################# PIECHART FOR DEVICES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager
%matplotlib inline
import itertools

fig, ax = plt.subplots()

plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['text.color'] = '#909090'
plt.rcParams['axes.labelcolor']= '#909090'
plt.rcParams['xtick.color'] = '#909090'
plt.rcParams['ytick.color'] = '#909090'
plt.rcParams['font.size']=12


from pathlib import Path
import re
tweets_trending_file_path = Path() / "saved_tweets.csv"
trending_tweets = pd.read_csv(tweets_trending_file_path)


import re
client_system = trending_tweets["client_platform"].tolist()


# REGULAR EXPRESSION IS USED ON EVERY DATA POINT IN DF TO GET THE CLIENT PLATFORM OUT OF THE 
# HTML ANCHOR TAG USED BY TWITTER FOR SHOWING CLIENT DEVICE

twitter_client_system = []
for i in client_system:
    match = re.search(r'Android|iPhone|Web App|Tweetdeck|TweetDeck|Tweetbot',i)
    if match is not None:
      device = match.group()
      twitter_client_system.append(device)

counts = dict()
for i in twitter_client_system:
  counts[i] = counts.get(i, 0) + 1


labels = list(counts.keys())
data = list(counts.values())
colors = sns.color_palette('pastel')[0:5]
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
plt.title("Device percentage used in trend", size= 10)        
plt.legend(labels)                   
plt.show()


In [ ]:
################ DEVICES BARPLOT

sns.barplot(x=labels,y=data)
plt.xlabel("Client Platform", size=10)
plt.ylabel("Number of devices", size=10)
plt.title("Device percentage used in trend", size= 15)                   
plt.show()

In [ ]:
################# PIECHART FOR DEVICES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager
%matplotlib inline
import itertools

fig, ax = plt.subplots()
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['text.color'] = '#909090'
plt.rcParams['axes.labelcolor']= '#909090'
plt.rcParams['xtick.color'] = '#909090'
plt.rcParams['ytick.color'] = '#909090'
plt.rcParams['font.size']=12


from pathlib import Path
import re
tweets_trending_file_path = Path() / "saved_tweets.csv"
trending_tweets = pd.read_csv(tweets_trending_file_path)


import re
verified_user_tweets = trending_tweets["verified"].tolist()

counts = dict()
for i in verified_user_tweets:
  counts[i] = counts.get(i, 0) + 1


labels = list(counts.keys())
legends = []
for label in labels:
    if label == True:
        legends.append("Verified")
    else:
        legends.append("Not Verified")
data = list(counts.values())
colors = sns.color_palette('pastel')[0:5]
plt.pie(data, labels = legends, colors = colors, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)
plt.title("Device percentage used in trend", size= 10)   
plt.legend(labels=legends)                    
plt.show()


In [ ]:
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

path_to_file = Path() / "saved_tweets.csv"

twitter_data = pd.read_csv(path_to_file)
tweets = twitter_data[["text"]]


tweets = twitter_data["text"].to_list()

document = ""
for tweet in tweets:
    document += tweet


document



tweet_tokens = nltk.word_tokenize(document)

stopwords = set(stopwords.words("english"))

tweet_tokens


filtered_tweet_text = [w for w in tweet_tokens if w not in stopwords]

wordcloud = WordCloud()

tweet_freq_dist = nltk.FreqDist(filtered_tweet_text)

sorted(tweet_freq_dist,key=tweet_freq_dist.__getitem__,reverse=True)

large_words = dict([(k,v) for (k,v) in tweet_freq_dist.items() if len(k) > 3])

frequency_dist = nltk.FreqDist(large_words)
frequency_dist.plot(30,cumulative=False)

wordcloud = WordCloud(max_font_size=50, max_words=100,background_color="black").generate_from_frequencies(frequency_dist)

plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()








In [ ]:
results = twitter.search(q=top_trend_on_twitter_query, count=100)

r = results["statuses"]

tweet_locations = [tweet for tweet in r if tweet["place"] is not None]

with open("sample.json", "w") as file:
    json.dump(r,file)

top_trend_on_twitter_query


In [ ]:
############### STREAMING API

############### ALTHOUGH STREAMING WAS USED FOR TESTING LOCATION TWEETS 
#### BUT THE TWEETS FROM STREAMS AREN"T USED THIS IS HERE JUST FOR REFERENCE PURPOSES 

from twython import TwythonStreamer
import csv
import json
import time



# counter = 0

def process_tweet(data):
        
        tweet = {}
        tweet["text"] = data["text"]
        tweet["created_at"] = data["created_at"]
        tweet["favorite_count"] = data["favorite_count"]
        if "place" in data and data["place"] != None:
            tweet["long"] = data["place"]["bounding_box"]["coordinates"][0][0][0]
            tweet["lat"] = data["place"]["bounding_box"]["coordinates"][0][0][1]
            print(tweet["long"], tweet["lat"], tweet["favorite_count"])
        else:
            tweet["long"] = None
            tweet["lat"] = None
        print(tweet)
        return tweet


def csv_set_header(data):
        with open("live_saved_tweets.csv", "a", encoding="utf-8",newline="") as file:
            header = data
            csv_writer = csv.writer(file)
            print("hello")
            csv_writer.writerow(header)

counter = 0

class MyStreamer(TwythonStreamer):
    end_time = time.time() + 10
    
    def on_success(self, data):
        global counter 
        if data["lang"] == "en":
            tweet_data = process_tweet(data)
            print(counter)
            
            
            if tweet_data["long"] is not None and tweet_data["lat"] is not None:
                self.save_to_csv(tweet_data)
                counter += counter
                print(counter)
            if counter >= 3:
                self.disconnect()


    def save_to_csv(self, tweet):
        with open(r'live_saved_tweets.csv', "a", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))





stream = MyStreamer(credentials["API_KEY"], credentials["API_SECRET"], credentials["ACCESS_TOKEN"], credentials["ACCESS_TOKEN_SECRET"])
csv_set_header(["text","created_at","favorite_count","long","lat"])



In [ ]:
##############   GETTING TWITTER COORDINATES

trending_tweets = pd.read_csv("saved_tweets.csv")
trending_tweets = trending_tweets.dropna()

import gmplot 
import webbrowser
import folium
from folium.plugins import MarkerCluster


########### UNCOMMENT BELOW CODE TO PLOT MAP USING GMPLOT
# usa_map =  gmplot.GoogleMapPlotter(53.81604806664296, -3.0548307614209813, 18 )
# usa_map.draw("/users/usama/desktop/usa.html")
# webbrowser.open_new_tab("/users/usama/desktop/usa.html")

map = folium.Map(location=[48, -102], zoom_start=4)


marker_cluster = MarkerCluster().add_to(map)
locationlist = trending_tweets[["lat", "long"]].values.tolist()
locationlist = locationlist[1:-1]
print(locationlist)
for point in range(0,len(locationlist)):
    folium.Marker(locationlist[point], popup=trending_tweets["location"].values.tolist()[point],
                icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')
                ).add_to(marker_cluster)


#print(top_trend_on_twitter_query)
map

####### FAKE TWEETS
# BOUNDING BOX SHOWS EXACT LOCATION, WHILE NAME SHOWS WHAATA USER HAVE PUT IN PROFILE



In [ ]:
################## STREAMER TREND 
# top_trend_on_twitter_query

In [ ]:
############################# For popular tweets location

# params = {}
# popular_tweets = Twitter_Utility()
# popular_tweets.search_tweet({params["q"]:top_trend_on_twitter_query,
#                                 params["count"]:100, 
#                                 params["result_type"]:"popular",
#                                 params["has:geo"]:True})
# popular_tweets.save_tweets()

In [ ]:

############### BOX PLOT FOR DEVICES & TWEETS
import seaborn as sns_1
df_tweets = pd.read_csv("saved_tweets.csv")
pattern = r'Android|iPhone|Web App|Tweetdeck|TweetDeck|Tweetbot'

filter=  df_tweets['client_platform'].str.contains(pattern)
df_tweets = df_tweets[filter]

df_tweets["clients_platform"] = twitter_client_system

ax2 = sns_1.boxplot(x="clients_platform", y="retweet_count" ,data=df_tweets)


In [ ]:
################ RELAION BETWEETN RETWEEET VS LIKES

sns.relplot(x="retweet_count", y="favorite_count",
             data=df_tweets)
plt.xlabel("Number of Retweets", size=10)
plt.ylabel("Number of Likes", size=10)
plt.title("Relation of Retweets Vs Likes", size= 15)                       
plt.show()

In [ ]:
ax = sns.boxplot(x="clients_platform", y="favorite_count" ,data=df_tweets)


In [ ]:
sns.distplot(df_tweets["retweet_count"],kde=True)

plt.xlabel("Number of Retweets", size=10)
plt.ylabel("Retweet Count", size=10)
plt.title("Distribution Of Retweet Count", size= 15)                       
plt.show()

In [ ]:
sns.jointplot(x="retweet_count",y="favorite_count" ,data=df_tweets, kind="reg")

In [ ]:
# sns.pairplot(df_tweets)

In [ ]:
sns.rugplot(df_tweets["favorite_count"])

In [ ]:
sns.rugplot(df_tweets["followers_count"])

In [ ]:
sns.scatterplot(x="followers_count", y="favorite_count", data=df_tweets)

plt.xlabel("Follower Count", size=10)
plt.ylabel("Number of Likes", size=10)
plt.title("Relation of Followers Vs Likes", size= 15)                       
plt.show()

In [ ]:
#####################  ASESSMENT 3 ###############################

top_trend_on_twitter_query = "Liverpool"
 
from pathlib import Path
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
import nltk 
# from sklearn.feature_extraction.text import CountVectorize

############### USE THIS FOR FIRST TIME TO DOWNLOAD VADER LEXICON
#import nltk
#nltk.download('vader_lexicon')

positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []


path_to_tweets = Path() / "saved_tweets.csv"
tweets = pd.read_csv(path_to_tweets)



tweets_ = tweets["text"].to_list()
no_of_tweets = len(tweets_)


for tweet in tweets_:
    tweet_list.append(tweet)
    analysis = TextBlob(tweet)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    polarity += analysis.sentiment.polarity
    if neg > pos:
        negative_list.append(tweet)
        negative += 1

    elif pos > neg:
         positive_list.append(tweet)
         positive += 1  

    elif pos == neg:
        neutral_list.append(tweet)
        neutral += 1  



def percentage(part,whole):
 return 100 * float(part)/float(whole)

positive = percentage(positive, no_of_tweets)
negative = percentage(negative, no_of_tweets)
neutral = percentage(neutral, no_of_tweets)
polarity = percentage(polarity, no_of_tweets)
    


positive = format(positive, ".1f")
negative = format(negative, ".1f")
neutral = format(neutral, ".1f")










In [ ]:
#################### WORDCLOUD UTILITY FUNCTION
import re


def wordcloud_generator(tweet_list, title, stopwords):
    document = ""
    for tweet in tweet_list:
        document += tweet


    document = re.sub("https"," ", document)
    tweet_tokens = nltk.word_tokenize(document)

    stopwords = set(stopwords)

    filtered_tweet_text = [w for w in tweet_tokens if w not in stopwords]
    wordcloud = WordCloud()

    tweet_freq_dist = nltk.FreqDist(filtered_tweet_text)

    sorted(tweet_freq_dist,key=tweet_freq_dist.__getitem__,reverse=True)

    large_words = dict([(k,v) for (k,v) in tweet_freq_dist.items() if len(k) > 3])

    frequency_dist = nltk.FreqDist(large_words)
    frequency_dist.plot(30,cumulative=False)

    wordcloud = WordCloud(max_font_size=50, max_words=100,background_color="black").generate_from_frequencies(frequency_dist)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title(title, size= 15)  
    plt.axis("off")
    plt.show()





In [ ]:
##### GENERATE WORCLOUD FOR TWEETS

from nltk.corpus import stopwords

custom_stop_words = ["this","they","This","That","They","RT","rt"] 
stopwords = stopwords.words("english")

for i in custom_stop_words:
    stopwords.append(i)

wordcloud_generator(positive_list, "Positive Sentiment Tweets Wordcloud", stopwords)
wordcloud_generator(negative_list, "Negative Sentiment Tweets Wordcloud", stopwords)
wordcloud_generator(neutral_list, "Neutral Tweets Wordcloud",stopwords)

In [ ]:
# import pandas as pd


# tweet_list = pd.DataFrame(tweet_list)
# neutral_list = pd.DataFrame(neutral_list)
# negative_list = pd.DataFrame(negative_list)
# positive_list = pd.DataFrame(positive_list)
# print("total number: ",len(tweet_list))
# print("positive number: ",len(positive_list))
# print("negative number: ", len(negative_list))
# print("neutral number: ",len(neutral_list))



In [ ]:
labels = ["Positive ["+ str(positive)+ "%]" , 
          "Neutral ["+ str(neutral)+ "%]",
          "Negative ["+ str(negative)+ "%]"]

sizes = [positive, neutral, negative]
colors = ["yellowgreen", "blue","red"]
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use("default")
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= " +  top_trend_on_twitter_query)
plt.axis("equal")
plt.show()

In [ ]:
######################### BAR GRAPH

import seaborn as sns

labels = ["Positive ["+ str(positive)+ "%]" , 
          "Neutral ["+ str(neutral)+ "%]",
          "Negative ["+ str(negative)+ "%]"]

sizes = [positive, neutral, negative]
colors = ["yellowgreen", "blue","red"]

percentage_ = [float(size) for size in sizes] 
sns.barplot(x= labels,y=percentage_)
plt.style.use("default")
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= " +  top_trend_on_twitter_query)
# plt.axis("equal")
plt.show()

In [ ]:
import re

tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]



#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub("RT @\w+: "," ",x)
remove_links = lambda x: re.sub(r"http\S+", "", x)
tw_list["text"] = tw_list.text.map(remove_rt).map(remove_links)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)

In [ ]:
##################### SENTIMENT ANALYSIS
#Calculating Negative, Positive, Neutral and Compound valuestw_list[[‘polarity’, ‘subjectivity’]] = tw_list[‘text’].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list["text"].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    if neg > pos:
        tw_list.loc[index, "sentiment"] = "negative"
    elif pos > neg:
        tw_list.loc[index, "sentiment"] = "positive"
    else:
        tw_list.loc[index, "sentiment"] = "neutral"
        tw_list.loc[index, "neg"] = neg
        tw_list.loc[index, "neu"] = neu
        tw_list.loc[index, "pos"] = pos
        tw_list.loc[index, "compound"] = comp
 
 
tw_list.head(10)




In [ ]:
tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [ ]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=["Total","Percentage"])#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

In [ ]:
# create data for Pie Chart
pichart = count_values_in_column(tw_list,"sentiment")
names= pichart.index
size=pichart["Percentage"]
 
# Create a circle for the center of the plot
my_circle=plt.Circle( (0,0), 0.7, color="white")
plt.pie(size, labels=names, colors=["green","blue","red"])
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
##################### GETTING ARTICLES AND TEXT MINING

################## GET NEWS API SECRET KEYS

### LIVERPOOL

import pprint
import requests
from bs4 import BeautifulSoup
import csv
import re

secret = 'fe92f1b3aa6040159483b4644196ba1b'

#### DEFINE THE ENDPOINT
url = "https://newsapi.org/v2/everything?"



# Specify the query and number of returns
top_trend_on_twitter_query = "Liverpool"
parameters = {
    'q': top_trend_on_twitter_query, # query phrase
    'pageSize': 20,  # maximum is 100
    'apiKey': secret,
    'from': "2022-04-19", # your own API key
    'to': "2022-05-03" # your own API key
}


response = requests.get(url, params= parameters)



############# GET FIRST NINE ARTICLES
articles_list = response.json()["articles"][0:12]


article_urls = []

articles = []


###########  TRANSOFRMATION OF ARTICLES LIST INTO DESIRED FORM
for article in articles_list:
    articles.append({})

    ############# GET CURRENT INDEX TO OPERATE ON, IN THIS CASE IT WOULD BE EMPTY DICT
    current_index = len(articles) - 1
    
    ############# ADD PROPERTIES TO THE EMPTY DIC 
    articles[current_index]["newspaper"] = article["source"]["name"] 
    articles[current_index]["author"] = article["author"] 
    articles[current_index]["article_title"] = article["title"] 
    articles[current_index]["article_url"] = article["url"] 
    articles[current_index]["date_posted"] = article["publishedAt"] 

    article_urls.append(article["url"])




############## GET INDIVIDUAL ARTICLE FROM ITS RESPECTIVE WEBSITE
for index, article in enumerate(articles):
    url = article["article_url"]
    response = requests.get(url)
    soap = BeautifulSoup(response.text, "html.parser")
    
    ind = 0
    ############# EXTRACT Paragraph tags from the body
    article_description = ""
    for every_p in soap.select("body p"):
        article_description += every_p.getText()
        #if not bool(re.search("JS ad blocker", article_description)):
        articles[index]["article_description"] = article_description
         #   ind += ind




def csv_set_article_header(article):
          with open("articles.csv", "a", encoding="utf-8",newline="") as file:
            header = [key for key in article]
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)



def save_article_to_csv(tweet):
            with open("articles.csv", "a", encoding="utf-8", newline="") as file:
                csv_writer = csv.writer(file)
                tweet_content = [tweet[key] for key in tweet]
                csv_writer.writerow(tweet_content)
                return tweet_content

        
    #### FUNCTION TO PROCESS WHOLE FETCHING & SAVING OF TWEETS 
def save_articles(articles):
    pattern = r"Please enable JS and disable any ad blocker|Please|please|blocker|adblocker|ad block|disable"
   
    for article in articles:
        if not bool(re.search(pattern, article["article_description"], re.IGNORECASE)):
            save_article_to_csv(article)



csv_set_article_header(articles[0])
save_articles(articles=articles)
print(len(articles))

In [ ]:
####################### TOPIC MODELING



import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

def load_data(path, file_name):
    document_list = []
    titles = []

    print(os.path.join(path, file_name))
    articles_path = Path() / "articles.csv"
    articles = pd.read_csv(articles_path)

    article_descriptions = articles["article_description"].tolist()
    article_title = articles["article_title"].tolist()
    print(len(article_descriptions))
    for article_desc in article_descriptions:
        if isinstance(article_desc, str):
            #print(article_desc)
            text = article_desc.strip()
            document_list.append(text)

    for article_title in article_title:
        titles.append(article_title)       
    print("total number of documents: ", len(document_list))
    
    return document_list,titles




def pre_process_data(doc_set):
    tokenizer = RegexpTokenizer(r"\w+")
    en_stop = set(stopwords.words("english"))

    p_stemmer = PorterStemmer()

    texts = []

    for i in doc_set:
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        stopped_tokens = [i for i in tokens if i not in en_stop]


        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

        texts.append(stemmed_tokens)
    
    return texts




def prepare_corpus(doc_clean):
    dictionary = corpora.Dictionary(doc_clean)

    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    return dictionary, doc_term_matrix



def create_gensim_lsa_model(doc_clean, number_of_topics, words):
    dictionary, doc_term_matrix = prepare_corpus(doc_clean)

    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)
    #print(lsamodel.print_topics(num_topics=number_of_topics, num_words = words) )
    return lsamodel






number_of_topics = 7
words = 10
document_list,titles =load_data("", "articles.csv")
clean_texts = pre_process_data(document_list)
model = create_gensim_lsa_model(clean_texts,number_of_topics, words)


model

 

In [ ]:
################## WORDCLOUD ARTICLES

from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

def wordcloud_draw(document, stopwords):
    tweet_tokens = nltk.word_tokenize(document)
    stopwords = set(stopwords)

    tweet_tokens


    filtered_tweet_text = [w for w in tweet_tokens if w not in stopwords]

    wordcloud = WordCloud()

    tweet_freq_dist = nltk.FreqDist(filtered_tweet_text)

    sorted(tweet_freq_dist,key=tweet_freq_dist.__getitem__,reverse=True)

    large_words = dict([(k,v) for (k,v) in tweet_freq_dist.items() if len(k) > 3])

    frequency_dist = nltk.FreqDist(large_words)
    frequency_dist.plot(30,cumulative=False)

    wordcloud = WordCloud(max_font_size=50, max_words=100,background_color="white").generate_from_frequencies(frequency_dist)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


path_to_file = Path() / "articles.csv"

article_data = pd.read_csv(path_to_file)


# https://stackoverflow.com/questions/57237193/delete-rows-in-pandas-given-a-regex

m = ~article_data['article_description'].str.contains('JS disable ad blocker ')

article_data = article_data[m] 
articles = article_data["article_description"].to_list()


custom_stop_words = ["this","they","This","That","They","RT","rt"] 
stopwords = stopwords.words("english")

for i in custom_stop_words:
    stopwords.append(i)
for index, document in enumerate(articles):
    if isinstance(document,str):
        wordcloud_draw(document=document, stopwords=stopwords)
    


In [ ]:
#################### WORD COUNT OF ARTICLES  & TIME SERIES
import datetime
from dateutil.parser import parse

articles = pd.read_csv("articles.csv")

list_of_articles = articles["article_description"].to_list()
article_title = articles["article_title"].to_list()
articles_date_posted = articles["date_posted"].apply(lambda x : str(datetime.datetime.strftime(parse(x),"%Y-%m-%d %H:%M:%S"))).to_list()

articles_word_count_list = []

for index, article in enumerate(list_of_articles):
    article_word_count = len(article.split())
    articles_word_count_list.append(article_word_count)
    print(article_title[index] + " " +"Word Count: " + str(article_word_count) + "\n")
    






import matplotlib.pyplot as plt
import datetime
import numpy as np

x = articles_date_posted
y = articles_word_count_list

plt.plot(x,y)
plt.xticks(rotation=75)
plt.title("Time Series Of Words Used in Articles Related To Liverpool")
plt.xlabel("Time", size=10)
plt.ylabel("Word Count", size=10)
plt.show()





In [11]:
###################### PLOT TOPICS
import re
import numpy as np
import pandas as  pd
import pyLDAvis 
import matplotlib.pyplot as plt
import spacy
import gensim

from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models as gensimvis
%matplotlib inline

df = pd.read_csv("articles.csv")


# CONVERT TO LIST
data = df["article_description"].to_list()  



def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
#print(data_words[:1])



# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])
stopwords = set(stopwords.words("english"))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out





# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)




# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)

pyLDAvis.save_html(vis, 'lda.html')

C:\Users\usama\Desktop\BCU\anaconda\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


NameError: name 'text' is not defined

In [ ]:
##################### ARTICLE SUMMARISATION
#https://medium.com/analytics-vidhya/simple-text-summarization-using-nltk-eedc36ebaaf8


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize 
from string import punctuation
from heapq import nlargest


articles = pd.read_csv("articles.csv")

list_of_article_desc = articles["article_description"].to_list()
article = list_of_article_desc[0]

stop_words = stopwords.words('english')
tokens = word_tokenize(article)
sent_token = sent_tokenize(article)

word_frequencies = {}
for word in tokens:    
    if word.lower() not in stop_words:
        if word.lower() not in punctuation:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1


max_frequency = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency


sentence_scores = {}
for sent in sent_token:
    sentence = sent.split(" ")
    for word in sentence:        
        if word.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.lower()]





select_length = int(len(sent_token)*0.05)


summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
final_summary = [word for word in summary]
summary = ' '.join(final_summary)

summary







In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords



stopwords = set(stopwords.words("english"))
articles = pd.read_csv("articles.csv")
articles_description_list = articles["article_description"].to_list()

article = articles_description_list[0]


article_text = re.sub("\d+", "", article)

article_text = re.sub(r'[0-9]', "", article_text)
article_text = ''.join([i for i in article_text if not i.isdigit()])
# print(article_text)
article_text = article_text.split(" ")
article_text =  [word for word in article_text if word not in stop_words]
# article_text = [word for word in article_text if len(word) > 3]


# articles_description_list_ = []
# for article in articles_description_list:
#     article_text = re.sub("[^A-Za-z0-9 ]|[0-9]|[0-9^[a-z]|[0-9$[a-z]]]+", "", article)
#     article_text = article_text.split(" ")
#     article_text = [word for word in article_text if len(word) > 3]
#     article_text = ' '.join(article_text)
#     articles_description_list_.append(article_text)


count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

#bag_of_words = count_vectorizer.fit_transform(articles_description_list_)
#tfidf = tfidf_vectorizer.fit_transform(articles_description_list_)

bag_of_words = count_vectorizer.fit_transform(article_text)
tfidf = tfidf_vectorizer.fit_transform(article_text)


feature_names = count_vectorizer.get_feature_names()
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

df_idf = pd.DataFrame(tfidf.toarray(), columns = tfidf_feature_names)
df_bow = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)


print(df_idf)
print(df_bow)

#df_bow.to_csv("bow.csv")
#df_bow.to_csv("tfidf.csv")









<>:15: DeprecationWarning: invalid escape sequence \d
<>:15: DeprecationWarning: invalid escape sequence \d
C:\Users\usama\AppData\Local\Temp/ipykernel_6884/1598887390.py:15: DeprecationWarning: invalid escape sequence \d
  article_text = re.sub("\d+", "", article)
